<a href="https://colab.research.google.com/github/aimbesi1/CSC_4850-MachineLearning-Project/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Default imports

%matplotlib inline

import numpy as np
import pandas as pd
import sklearn 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Specific imports
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline, make_pipeline

Using the housing dataset from Kaggle: https://www.kaggle.com/datasets/mirbektoktogaraev/madrid-real-estate-market

In [2]:
### Manually download it and upload to this istance data sample space
### Note DO NOT change these operations or all your answers will be incorrect

### Let's do some transformations and extra features on this.
df=pd.read_csv('houses_Madrid.csv', encoding='utf-8')

In [3]:
# Select/clean features here
df.describe()
# Do what needs to be done to the data so that the models can run

,Unnamed: 0,id,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,...,rent_price_by_area,buy_price,buy_price_by_area,built_year,are_pets_allowed,is_furnished,is_kitchen_equipped,has_private_parking,has_public_parking,parking_price
count,21742.000000,21742.000000,21616.000000,8228.000000,21742.000000,21726.000000,1437.000000,1432.000000,0.0,0.0,...,0.0,2.174200e+04,21742.000000,10000.000000,0.0,0.0,0.0,0.0,0.0,7719.000000
mean,10870.500000,10871.500000,146.920892,103.458192,3.005749,2.091687,3.128740,241.692737,NaN,NaN,...,NaN,6.537356e+05,4020.523871,1970.046400,NaN,NaN,NaN,NaN,NaN,2658.000518
std,6276.519112,6276.519112,134.181865,88.259192,1.510497,1.406992,0.907713,247.484853,NaN,NaN,...,NaN,7.820821e+05,1908.418774,69.386705,NaN,NaN,NaN,NaN,NaN,13360.966258
min,0.000000,1.000000,13.000000,1.000000,0.000000,1.000000,1.000000,1.000000,NaN,NaN,...,NaN,3.600000e+04,447.000000,1723.000000,NaN,NaN,NaN,NaN,NaN,0.000000
25%,5435.250000,5436.250000,70.000000,59.000000,2.000000,1.000000,2.000000,2.000000,NaN,NaN,...,NaN,1.980000e+05,2551.000000,1957.000000,NaN,NaN,NaN,NaN,NaN,0.000000
50%,10870.500000,10871.500000,100.000000,79.000000,3.000000,2.000000,3.000000,232.000000,NaN,NaN,...,NaN,3.750000e+05,3720.000000,1970.000000,NaN,NaN,NaN,NaN,NaN,0.000000
75%,16305.750000,16306.750000,162.000000,113.000000,4.000000,2.000000,4.000000,354.000000,NaN,NaN,...,NaN,7.636000e+05,5000.000000,1994.000000,NaN,NaN,NaN,NaN,NaN,0.000000
max,21741.000000,21742.000000,999.000000,998.000000,24.000000,16.000000,7.000000,997.000000,NaN,NaN,...,NaN,8.800000e+06,18889.000000,8170.000000,NaN,NaN,NaN,NaN,NaN,600000.000000


In [5]:
# Update X values later
df_short = df.iloc[:4000, :][['sq_mt_built', 'rent_price', 'buy_price']].dropna()

# df_short['high_price'] = df_short['buy_price'].apply(lambda x: 1 if x > np.median(df_short['buy_price']) else 0)
X = df_short[['sq_mt_built', 'rent_price']]
y = df_short['buy_price']
# y = df_short['high_price'].values
df_short.describe()



,sq_mt_built,rent_price,buy_price
count,4000.000000,4.000000e+03,4.000000e+03
mean,98.567000,-8.281727e+03,2.997627e+05
std,54.966729,5.470396e+05,3.118178e+05
min,16.000000,-3.459028e+07,3.600000e+04
25%,65.000000,7.250000e+02,1.490000e+05
50%,85.000000,9.700000e+02,2.210000e+05
75%,115.000000,1.218000e+03,3.232500e+05
max,700.000000,2.517000e+03,8.800000e+06


In [5]:
# def discretize(y_subset, y, n):
#   quantiles = []
#   i = 0
#   while i <= n:
#     quantiles.append(y.quantile(i / n))
#     i = i + 1
#   return pd.cut(y_subset, bins=quantiles, labels=range(n))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.50, random_state=1234)

y_train_d = pd.cut(y_train, bins=[0, y.quantile(0.25), y.quantile(0.5), y.quantile(0.75), y.quantile(1)], labels=[0, 1, 2, 3])
y_test_d = pd.cut(y_test, bins=[0, y.quantile(0.25), y.quantile(0.5), y.quantile(0.75), y.quantile(1)], labels=[0, 1, 2, 3])

y_train_d.loc[y_train_d.isna()].head()

Series([], Name: buy_price, dtype: category
Categories (4, int64): [0 < 1 < 2 < 3])

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import svm
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

def init_models():
  dt = DecisionTreeClassifier(random_state=1234)
  perceptron = Perceptron(random_state=1234)
  nb = GaussianNB()
  log_reg = LogisticRegression(random_state=1234, solver='liblinear', multi_class='ovr', max_iter=150)
  lin_reg = LinearRegression()
  ridge = Ridge(alpha=0.5, fit_intercept=True, random_state=1234)
  lasso = Lasso(alpha=0.1, fit_intercept=True, random_state=1234)
  ef = ElasticNet(alpha=0.9, fit_intercept=True, random_state=1234)
  svm_linear = make_pipeline(StandardScaler(), svm.SVC(kernel="linear", random_state=1234))
  svm_rbf = make_pipeline(StandardScaler(), svm.SVC(kernel="rbf", random_state=1234))
  gb = HistGradientBoostingClassifier(random_state=1234)
  mlp = make_pipeline(StandardScaler(), MLPClassifier(solver='lbfgs', random_state=1234))
  # svr_poly = svm.SVR(kernel="poly")

  c_dict = {
      "dt": dt,
      "perceptron": perceptron,
      "nb": nb,
      "log_reg": log_reg,
      "svm_linear": svm_linear,
      "svm_rbf": svm_rbf,
      # "gb": gb,
      # "mlp": mlp,
  }

  r_dict = {
      "lin_reg": lin_reg,
      "ridge": ridge,
      "lasso": lasso,
      "ef": ef,
  }

  return c_dict, r_dict

In [11]:

# Train Classifiers
def train_classifier(models, X_train, X_test, y_train, y_test, lc_dict, m_dict, b_dict):
  def get_metrics(model, X_train, y_train):
    # lc = learning_curve(estimator=model, X=X_train, y=y_train, cv=10, shuffle=True, random_state=1234)
    lc = -1
    scorer = {
        "precision": metrics.make_scorer(metrics.precision_score, average="micro"),
        "recall": metrics.make_scorer(metrics.precision_score, average="micro"),
        "accuracy": metrics.make_scorer(metrics.precision_score, average="micro")
    }
    cv = cross_validate(model, X_train, y_train, cv=10, scoring=scorer, return_train_score=False, return_estimator=True)
    best_score_index = cv["test_accuracy"].argmax()
    best = cv["estimator"][best_score_index]
    return lc, cv, best

  for key in models.keys():
    # if key in {"svm_linear", "gb", "mlp"}:
    #   print(f"Found key {key}, scaling data")
    #   X_train_scaled = StandardScaler().fit_transform(X_train)
    #   X_test_scaled = StandardScaler().fit_transform(X_test)

    #   models[key].fit(X_train_scaled, y_train)
    #   # predY = models[key].predict(X_test_scaled)
    #   lc, cv, best = get_metrics(models[key], X_train_scaled, y_train)
    # else:
    models[key].fit(X_train, y_train)
    # predY = models[key].predict(X_test)
    lc, cv, best = get_metrics(models[key], X_train, y_train)
    
    # lc_dict[key] = lc
    m_dict[key] = cv
    b_dict[key] = best

# Test Classifiers
def test_classifier(models, X_train, X_test, y_train, y_test, lc_dict, bp_dict):
  for key in models.keys():
  #   if key in {"svm_linear", "gb", "mlp"}:
  #     print(f"Found key {key}, scaling data")
  #     X_train_scaled = StandardScaler().fit_transform(X_train)
  #     X_test_scaled = StandardScaler().fit_transform(X_test)

  #     models[key].fit(X_train_scaled, y_train)
  #     predY = models[key].predict(X_test_scaled)
  #     lc = learning_curve(estimator=models[key], X=X_test, y=y_test, shuffle=True, random_state=1234)
  #   else:
    models[key].fit(X_train, y_train)
    predY = models[key].predict(X_test)
    lc = learning_curve(estimator=models[key], X=X_test, y=y_test, shuffle=True, random_state=1234)

    lc_dict[key] = lc
    bp_dict[key] = predY


def train_regression(models, X_train, X_test, y_train, y_test, lc_dict, m_dict, b_dict):
  def get_metrics(model, X_train, y_train):
    # lc = learning_curve(estimator=model, X=X_train, y=y_train, cv=10, shuffle=True, random_state=1234)
    lc = -1
    cv = cross_validate(model, X_train, y_train, cv=10, return_train_score=False, return_estimator=True)
    # best_score_index = cv["test_accuracy"].argmax()
    # best = cv["estimator"][best_score_index]
    best = None
    return lc, cv, best


  for key in models.keys():
    lc, cv, best = None, None, None
    if key in {"svm_lin", "lin_reg", "ridge", "lasso", "ef"}:
      X_train_scaled = StandardScaler().fit_transform(X_train)
      X_test_scaled = StandardScaler().fit_transform(X_test)

      models[key].fit(X_train_scaled, y_train)
      predY = models[key].predict(X_test_scaled)
      lc, cv, best = get_metrics(models[key], X_train_scaled, y_train)
    else:
      models[key].fit(X_train, y_train)
      predY = models[key].predict(X_test)
      lc, cv, best = get_metrics(models[key], X_train, y_train)
    
    # lc_dict[key] = lc
    m_dict[key] = cv
    b_dict[key] = best

def test_regression(models, X_train, X_test, y_train, y_test, lc_dict, bp_dict):
  for key in models.keys():
    lc, predY = None, None
    if key in {"svm_lin", "lin_reg", "ridge", "lasso", "ef"}:
      X_train_scaled = StandardScaler().fit_transform(X_train)
      X_test_scaled = StandardScaler().fit_transform(X_test)

      models[key].fit(X_train_scaled, y_train)
      predY = models[key].predict(X_test_scaled)
      lc = learning_curve(estimator=models[key], X=X_test_scaled, y=y_test, shuffle=True, random_state=1234)
    else:
      models[key].fit(X_train, y_train)
      predY = models[key].predict(X_test)
      lc = learning_curve(estimator=models[key], X=X_test, y=y_test, shuffle=True, random_state=1234)

    lc_dict[key] = lc
    bp_dict[key] = predY

  
  



# def run_models(models, X_train, y_train, X_test, y_test):

#   # Decision Tree
#   models["dt"].fit(X_train, y_train)
#   pred_Y1 = models["dt"].predict(X_test)
#   print(metrics.classification_report(y_test, pred_Y1))

#   # SVM Linear
#   f1_scaledX = StandardScaler().fit(X_train).transform(X_train)
#   f1_scaledX_test = StandardScaler().fit(X_test).transform(X_test)

#   models["svm_linear"].fit(f1_scaledX, y_train)
#   pred_Y1 = models["svm_linear"].predict(f1_scaledX_test)
#   print(metrics.classification_report(y_test, pred_Y1))

#   sizes1, train_scores1, valid_scores1 = learning_curve(estimator = svm_linear, X = f1_scaledX_test, y=y_test, shuffle = True, random_state = 1234)

#   # RBF
#   models["svm_rbf"].fit(X_train, y_train)
#   pred_Y3 = models["svm_rbf"].predict(X_test)
#   print(metrics.classification_report(y_test, pred_Y3))

#   sizes3, train_scores3, valid_scores3 = learning_curve(estimator = svm_rbf, X = X_test, y=y_test, shuffle = True, random_state = 1234)


 

def show_learning_curve(ax, lc_sizes, train_scores, valid_scores, title):
  train_mean = train_scores.mean(axis=1)
  valid_mean = valid_scores.mean(axis=0)
  ax.plot(lc_sizes, train_mean, label = "Training Error")
  ax.plot(lc_sizes, valid_mean, label = "Validation Error")
  ax.legend()
  ax.set_title(title)


# # Learning curves
# f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15, 5), sharey=True)
# show_learning_curve(ax1, sizes1, train_scores1, valid_scores1, "Linear")
# show_learning_curve(ax2, sizes2, train_scores2, valid_scores2, "Poly")
# show_learning_curve(ax3, sizes3, train_scores3, valid_scores3, "RBF")

# acc_linear = metrics.accuracy_score(fold["y_test"], pred_Y1)
# acc_poly = metrics.accuracy_score(fold["y_test"], pred_Y2)
# acc_rbf = metrics.accuracy_score(fold["y_test"], pred_Y3)
# return {"linear": svm_linear, "poly": svm_poly, "rbf": svm_rbf}

# f1_results = run_models(fold1, svm_linear, svm_poly, svm_rbf)

In [14]:
##### Pipeline

''' 
For each sampling:
  Set up the models
  Initialize L: list of LCs (list of training scores + valid scores)
  Initialize M: list of metrics (predicted Y)
  

'''


def pipe(X, y):
  train_sizes = [0.50, 0.70, 0.80]
  # learning_curves and metrics are arrays of 3 dictionaries each.
  # Each dictionary will hold the learning curve and metrics data for each model in each fold. 
  learning_curves = np.array([{}, {}, {}])
  metrics = np.array([{}, {}, {}])
  models = []
  best_models = np.array([{}, {}, {}])
  best_predictions = np.array([{}, {}, {}])

  for i in range(len(train_sizes)):
    models = init_models()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_sizes[i], random_state=1234)

    y_train_d = pd.cut(y_train, bins=[0, y.quantile(0.25), y.quantile(0.5), y.quantile(0.75), y.quantile(1)], labels=[0, 1, 2, 3])
    y_test_d = pd.cut(y_test, bins=[0, y.quantile(0.25), y.quantile(0.5), y.quantile(0.75), y.quantile(1)], labels=[0, 1, 2, 3])

    # y_train_d.describe()


    # Test with 1 model
    train_classifier(models[0], X_train, X_test, y_train_d, y_test_d, learning_curves[i], metrics[i], best_models[i])
    # train_regression(models[1], X_train, X_test, y_train, y_test, learning_curves[i], metrics[i], best_models[i])
    
    
    # print(learning_curves[i])
    print(f"Metrics for sample {i}:")
    print(metrics[i])

    print(f"Best model in sample {i}:")
    print(best_models[i])

    # Stop at 1 loop with break
    # break

    # Now train the best models on the whole training set
    
    test_classifier(models[0], X_train, X_test, y_train_d, y_test_d, learning_curves[i], best_predictions[i])
    # test_regression(models[1], X_train, X_test, y_train, y_test, learning_curves[i], best_predictions[i])

    print(f"Best prediction data for sample {i}:")
    print(best_predictions[i])

    
  
  # Learning curves
    f, ax = plt.subplots(1, len(models[0]), figsize = (15, 5), sharey=True)

    for key in models[0]:
      for j in range(3):
        show_learning_curve(ax[i], learning_curves[i])
      # break


  
  

    


# def pipe(X, y):
#   train_sizes = [0.50, 0.70, 0.80]
#   for size in train_sizes:
#     models = init_models()
#     rbf = models["svm_rbf"]
#     X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, random_state=1234)

#     kfold = KFold(n_splits=10, random_state=1234, shuffle=True)

#     for index, (train, test) in enumerate(kfold.split(X_train,y_train)):
#       X_train_folds = X_train[train]
#       y_train_folds = y_train[train]

#       X_test_folds = X_train[test]
#       y_test_folds = y_train[test]

#       run_models(models, X_train_folds, y_train_folds, X_test_folds, y_test_folds)

pipe(X, y)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Metrics for sample 0:
{'dt': {'fit_time': array([0.00368977, 0.00351858, 0.00386477, 0.00397587, 0.00381041,
       0.00743055, 0.00677347, 0.00520825, 0.00595069, 0.00422287]), 'score_time': array([0.00626826, 0.00747538, 0.00742221, 0.00630283, 0.01278186,
       0.00765061, 0.00700617, 0.01114917, 0.01181936, 0.00656366]), 'estimator': [DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234)], 'test_precision': array([0.995, 0.995, 0.99 , 1.   , 1.   , 0.995, 0.995, 0.995, 1.   ,
       1.   ]), 'test_recall': array([0.995, 0.995, 0.99 , 1.   , 1.   , 0.995, 0.995, 0.995, 1.   ,
       1.   ]), 'test_accuracy': array

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best prediction data for sample 0:
{'dt': array([1, 1, 2, ..., 3, 2, 0]), 'perceptron': array([1, 3, 3, ..., 3, 3, 3]), 'nb': array([1, 0, 2, ..., 3, 2, 0]), 'log_reg': array([0, 0, 0, ..., 0, 0, 0]), 'svm_linear': array([0, 0, 2, ..., 2, 2, 0]), 'svm_rbf': array([0, 0, 2, ..., 2, 2, 0])}


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Metrics for sample 1:
{'dt': {'fit_time': array([0.00418878, 0.00436234, 0.00432658, 0.00420046, 0.00413394,
       0.00389385, 0.00394177, 0.00391221, 0.00400138, 0.00382853]), 'score_time': array([0.00685954, 0.00671649, 0.00774741, 0.00907063, 0.00656533,
       0.00681305, 0.00649381, 0.00652242, 0.00646067, 0.00696635]), 'estimator': [DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234)], 'test_precision': array([0.99642857, 0.99642857, 1.        , 0.99642857, 0.99285714,
       1.        , 1.        , 0.99642857, 0.99642857, 1.        ]), 'test_recall': array([0.99642857, 0.99642857, 1.        , 0.99642857, 0.9

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Metrics for sample 2:
{'dt': {'fit_time': array([0.00425029, 0.00408435, 0.0043962 , 0.00420713, 0.00409412,
       0.00452447, 0.00451612, 0.00452042, 0.00439525, 0.00443244]), 'score_time': array([0.00663424, 0.00649786, 0.00661135, 0.00639606, 0.00723648,
       0.00735664, 0.00721955, 0.00716853, 0.00718498, 0.00734973]), 'estimator': [DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234), DecisionTreeClassifier(random_state=1234)], 'test_precision': array([0.996875, 1.      , 0.99375 , 1.      , 0.996875, 0.996875,
       1.      , 0.996875, 0.996875, 1.      ]), 'test_recall': array([0.996875, 1.      , 0.99375 , 1.      , 0.996875, 0.996875,
       1.  

In [9]:
def get_metrics(model):
  pred_Y = model.predict(X_test)
  acc = metrics.accuracy_score(y_test, pred_Y)
  precision = metrics.precision_score(y_test, pred_Y)
  recall = metrics.recall_score(y_test, pred_Y)
  f1_score = metrics.f1_score(y_test, pred_Y)
  rmse = np.sqrt(metrics.mean_squared_error(y_test, pred_Y))
  return [acc, precision, recall, f1_score, rmse]

# table = pd.DataFrame()
# table["Metric"] = ["Accuracy", "Precision", "Recall", "F1", "RMSE"]
# table["Linear - Fold 2"] = get_metrics(best_linear)
# table["Poly - Fold 2"] = get_metrics(best_poly)
# table["RBF - Fold 2"] = get_metrics(best_rbf)

# table.head()